In [1]:
import psycopg2
import random

#### Database connection settings

In [2]:
db_settings = {
    'flight': {
        'host': "localhost",
        'port': 5001,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
    'hotel': {
        'host': "localhost",
        'port': 5002,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
    'account': {
        'host': "localhost",
        'port': 5003,
        'database': "postgres",
        'user': "postgres",
        'password': "pgpass"
    },
}

#### Initialization functions

In [3]:
def get_rand_non_repetitve_int():
    rand_list = list()
    for i in range(10):
          number = random.randint(1,100)
          if number not in rand_list: 
              rand_list.append(number)
              yield number

def db_init(query, connection):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query)
            connection.commit()
            print('Table created')
        except Exception as e:
            print(e)
            connection.rollback()

def flight_init(connection):
    query = '''
    drop table if exists flight_booking;
    create table flight_booking(
        booking_id serial primary key,
        client_name varchar(255),
        flight_number  varchar(255),
        airpot_of_departure varchar(255),
        destination_airport varchar(255),
        date int
    );
    truncate table flight_booking;
    '''
    db_init(query, connection)

def hotel_init(connection):
    query = '''
    drop table if exists hotel_booking;
    create table hotel_booking(
        booking_id serial primary key,
        client_name varchar(255),
        hotel_name  varchar(255),
        check_in_date int,   
        check_out_date int
    );
    truncate table hotel_booking;
    '''
    db_init(query, connection)

def account_init(connection):
    query = '''
    drop table if exists account;
    create table account(
        account_id serial primary key,
        client_name varchar(255),
        deposit int check (deposit >= 0)
    );
    truncate table account;
    insert into account(client_name, deposit) 
    values 
    ('Dmytro Domin', 100)
    '''
    db_init(query, connection)

def add_reservation(client_name, flight_record, hotel_record, connections):
    flight_query = '''
        insert into flight_booking(client_name, flight_number, airpot_of_departure, destination_airport, date)
        values
        ('{}', '{}', '{}', '{}', {});
    '''.format(client_name, *flight_record)
    hotel_query = '''
        insert into hotel_booking(client_name, hotel_name, check_in_date, check_out_date)
        values
        ('{}', '{}', {}, {});
    '''.format(client_name, *hotel_record)
    account_query = '''
        update account set deposit = deposit - 35 WHERE client_name = '{}';
    '''.format(client_name)
    
    
    transaction_id = connections['flight'].xid(next(get_rand_non_repetitve_int()),'abc','de')
    
    try:
        cursors = dict()
        for connection_name, connection in connections.items():
            connection.tpc_begin(transaction_id)
            cursors.update({connection_name: connection.cursor()})
        
        if 'flight' in cursors.keys():
            cursors['flight'].execute(flight_query)
        if 'hotel' in cursors.keys():
            cursors['hotel'].execute(hotel_query)
        if 'account' in cursors.keys():
            cursors['account'].execute(account_query)

        for connection_name, connection in connections.items():
            connection.tpc_prepare()

        for connection_name, connection in connections.items():
            connection.tpc_commit()

        return transaction_id, True

    except Exception as e:
        print(e)
        for connection_name, connection in connections.items():
                    connection.tpc_rollback()
        return transaction_id, False

### Working with flight and hotel databases on two phase commit 

In [4]:
connections = db_connections = {
    'flight': psycopg2.connect(**db_settings['flight']),
    'hotel': psycopg2.connect(**db_settings['hotel']),
    }
    
flight_init(connections['flight'])
hotel_init(connections['hotel'])

Table created
Table created


In [5]:
client_name = 'Ivan Baran'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, connections['hotel'].cursor() as hotel_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

(<Xid: (100, 'abc', 'de')>, True)
flight_booking table records:
(1, 'Ivan Baran', 'FR3069', 'KBP', 'BCN', 20220115)
hotel_booking table records:
(1, 'Ivan Baran', 'Radisson', 20220115, 20220121)


In [6]:
client_name = 'Stepan Baran'
flight_booking = ('FR7262', 'KRK', 'KBP', 20220115)
hotel_booking = ('Premier', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, connections['hotel'].cursor() as hotel_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

(<Xid: (90, 'abc', 'de')>, True)
flight_booking table records:
(1, 'Ivan Baran', 'FR3069', 'KBP', 'BCN', 20220115)
(2, 'Stepan Baran', 'FR7262', 'KRK', 'KBP', 20220115)
hotel_booking table records:
(1, 'Ivan Baran', 'Radisson', 20220115, 20220121)
(2, 'Stepan Baran', 'Premier', 20220115, 20220121)


In [7]:
client_name = 'Petro Baran'
flight_booking = ('PQ482', 'DXB', 'KBP', 20220115)
hotel_booking = ('Hiltons', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, connections['hotel'].cursor() as hotel_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

(<Xid: (5, 'abc', 'de')>, True)
flight_booking table records:
(1, 'Ivan Baran', 'FR3069', 'KBP', 'BCN', 20220115)
(2, 'Stepan Baran', 'FR7262', 'KRK', 'KBP', 20220115)
(3, 'Petro Baran', 'PQ482', 'DXB', 'KBP', 20220115)
hotel_booking table records:
(1, 'Ivan Baran', 'Radisson', 20220115, 20220121)
(2, 'Stepan Baran', 'Premier', 20220115, 20220121)
(3, 'Petro Baran', 'Hiltons', 20220115, 20220121)


In [8]:
for connection in connections.values():
    connection.close()

### Working with flight, hotel and account databases on two phase commit 

In [9]:
connections = db_connections = {
    'flight': psycopg2.connect(**db_settings['flight']),
    'hotel': psycopg2.connect(**db_settings['hotel']),
    'account': psycopg2.connect(**db_settings['account']),
    }
    
flight_init(connections['flight'])
hotel_init(connections['hotel'])
account_init(connections['account'])

Table created
Table created
Table created


In [10]:
client_name = 'Dmytro Domin'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, \
     connections['hotel'].cursor() as hotel_cursor, \
     connections['account'].cursor() as account_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')
        account_cursor.execute('select * from account;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()
        account_records = account_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
        connections['account'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()
        connections['account'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

print('account table records:')
for row in account_records:
    print(row)

(<Xid: (26, 'abc', 'de')>, True)
flight_booking table records:
(1, 'Dmytro Domin', 'FR3069', 'KBP', 'BCN', 20220115)
hotel_booking table records:
(1, 'Dmytro Domin', 'Radisson', 20220115, 20220121)
account table records:
(1, 'Dmytro Domin', 65)


In [11]:
client_name = 'Dmytro Domin'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, \
     connections['hotel'].cursor() as hotel_cursor, \
     connections['account'].cursor() as account_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')
        account_cursor.execute('select * from account;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()
        account_records = account_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
        connections['account'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()
        connections['account'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

print('account table records:')
for row in account_records:
    print(row)

(<Xid: (51, 'abc', 'de')>, True)
flight_booking table records:
(1, 'Dmytro Domin', 'FR3069', 'KBP', 'BCN', 20220115)
(2, 'Dmytro Domin', 'FR3069', 'KBP', 'BCN', 20220115)
hotel_booking table records:
(1, 'Dmytro Domin', 'Radisson', 20220115, 20220121)
(2, 'Dmytro Domin', 'Radisson', 20220115, 20220121)
account table records:
(1, 'Dmytro Domin', 30)


In [12]:
client_name = 'Dmytro Domin'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

transaction_id = add_reservation(client_name, flight_booking, hotel_booking, connections)
print(transaction_id)

with connections['flight'].cursor() as flight_cursor, \
     connections['hotel'].cursor() as hotel_cursor, \
     connections['account'].cursor() as account_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')
        account_cursor.execute('select * from account;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()
        account_records = account_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
        connections['account'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()
        connections['account'].rollback()

print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

print('account table records:')
for row in account_records:
    print(row)

new row for relation "account" violates check constraint "account_deposit_check"
DETAIL:  Failing row contains (1, Dmytro Domin, -5).

(<Xid: (45, 'abc', 'de')>, False)
flight_booking table records:
(1, 'Dmytro Domin', 'FR3069', 'KBP', 'BCN', 20220115)
(2, 'Dmytro Domin', 'FR3069', 'KBP', 'BCN', 20220115)
hotel_booking table records:
(1, 'Dmytro Domin', 'Radisson', 20220115, 20220121)
(2, 'Dmytro Domin', 'Radisson', 20220115, 20220121)
account table records:
(1, 'Dmytro Domin', 30)


In [13]:
for connection in connections.values():
    connection.close()

### Account table lock demo

In [14]:
def add_reservation_account_lock(client_name, flight_record, hotel_record, connections):
    flight_query = '''
        insert into flight_booking(client_name, flight_number, airpot_of_departure, destination_airport, date)
        values
        ('{}', '{}', '{}', '{}', {});
    '''.format(client_name, *flight_record)
    hotel_query = '''
        insert into hotel_booking(client_name, hotel_name, check_in_date, check_out_date)
        values
        ('{}', '{}', {}, {});
    '''.format(client_name, *hotel_record)
    account_query = '''
        update account set deposit = deposit - 35 WHERE client_name = '{}';
    '''.format(client_name)
    
    
    transaction_id = connections['flight'].xid(next(get_rand_non_repetitve_int()),'abc','de')
    
    try:
        cursors = dict()
        for connection_name, connection in connections.items():
            connection.tpc_begin(transaction_id)
            cursors.update({connection_name: connection.cursor()})
        
        if 'flight' in cursors.keys():
            cursors['flight'].execute(flight_query)
        if 'hotel' in cursors.keys():
            cursors['hotel'].execute(hotel_query)
        if 'account' in cursors.keys():
            cursors['account'].execute(account_query)

        for connection_name, connection in connections.items():
            connection.tpc_prepare()

        for connection_name, connection in connections.items():
            # Changes go here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            if connection_name != 'account':
                connection.tpc_commit()

        return transaction_id, True

    except Exception as e:
        print(e)
        for connection_name, connection in connections.items():
                    connection.tpc_rollback()
        return transaction_id, False

In [15]:
connections = db_connections = {
    'flight': psycopg2.connect(**db_settings['flight']),
    'hotel': psycopg2.connect(**db_settings['hotel']),
    'account': psycopg2.connect(**db_settings['account']),
    }
    
flight_init(connections['flight'])
hotel_init(connections['hotel'])
account_init(connections['account'])

Table created
Table created
Table created


In [16]:
client_name = 'Dmytro Domin'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

# Chanhes go here !!!!!!!!!!!!!!!!!!!!!!
add_reservation_account_lock(client_name, flight_booking, hotel_booking, connections)

(<Xid: (90, 'abc', 'de')>, True)

In [51]:
client_name = 'Dmytro Domin'
flight_booking = ('FR3069', 'KBP', 'BCN', 20220115)
hotel_booking = ('Radisson', 20220115, 20220121)

# Chanhes go here !!!!!!!!!!!!!!!!!!!!!!
add_reservation_account_lock(client_name, flight_booking, hotel_booking, connections)

(<Xid: (28, 'abc', 'de')>, True)

In [55]:
connections['account'].tpc_recover()

ProgrammingError: tpc_recover cannot be used with a prepared two-phase transaction

In [54]:
with connections['flight'].cursor() as flight_cursor, \
     connections['hotel'].cursor() as hotel_cursor, \
     connections['account'].cursor() as account_cursor:
    try:
        flight_cursor.execute('select * from flight_booking;')
        hotel_cursor.execute('select * from hotel_booking;')
        account_cursor.execute('select * from account;')

        flight_records = flight_cursor.fetchall()
        hotel_records = hotel_cursor.fetchall()
        account_records = account_cursor.fetchall()

        connections['flight'].commit()
        connections['hotel'].commit()
        connections['account'].commit()
    except Exception as e:
        print(e)
        connections['flight'].rollback()
        connections['hotel'].rollback()
        connections['account'].rollback()
        
print('flight_booking table records:')
for row in flight_records:
    print(row)

print('hotel_booking table records:')
for row in hotel_records:
    print(row)

print('account table records:')
for row in account_records:
    print(row)

execute cannot be used with a prepared two-phase transaction


ProgrammingError: rollback cannot be used during a two-phase transaction

![alt text](screenshot_pending_transaction.png "Pending transaction")
![alt text](screenshot_rollback_prepared.png "Rollback prepared")